In [1]:
import sys
sys.path.append('../')
from src import optimizer as _optimizer
from src import visualization as _visualization
from src import metrics as _metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 
import seaborn as sns

sns.set(rc={'figure.figsize':(20,8)})
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
pd.set_option('display.max_rows', 120)

## Reading / Displaying the Data 

In [2]:
df_products = pd.read_csv('../data/products_features_exp4.csv')
df_products = df_products.set_index('code')
df_products.head()

,cat_code,source_channel_quantity_152bf0ce464047b9499ccb9e5b9b77a8,source_channel_quantity_3ab2427543039f8c9f17d06f6f65a3a7,source_channel_quantity_5a97b8efd901c1d28ff86522b95babb9,source_channel_quantity_67c19e107de33cab7ea9a9db8bc9ccd2,source_channel_quantity_7261d300057219056592010c7bdaf5ee,source_channel_quantity_98defd6ee70dfb1dea416cecdf391f58,source_channel_quantity_9d3e0fcbc1f16d80a76026e8f1c26002,source_channel_quantity_af082bb0c2fa1414655017d464aa0262,source_channel_quantity_ea2912716be1999ab62d5b9dfa4f58f9,...,month_quantity_9,month_quantity_10,month_quantity_11,month_quantity_12,perc_pis_cofins_quantity_0.0,perc_pis_cofins_quantity_9.25,weekend_quantity_0,weekend_quantity_1,kmeans_cluster,hc_cluster
code,,,,,,,,,,,,,,,,,,,,,
e6762ba2ffbca07ab6cee7551caeaad5,1,0.048780,0.0,0.000000,0.0,0.02439,0.048780,0.146341,0.000000,0.000000,...,0.02439,0.268293,0.073171,0.121951,0.000000,1.000000,0.756098,0.243902,4,9
d408e1b5e841dde4e15a4cfa182e3812,5,0.013333,0.0,0.026667,0.0,0.00000,0.040000,0.040000,0.000000,0.026667,...,0.00000,0.000000,0.186667,0.160000,0.000000,1.000000,0.746667,0.253333,8,6
29424aaf6e27a8dbe4b7273a0a39131d,5,0.034091,0.0,0.000000,0.0,0.00000,0.017045,0.051136,0.000000,0.051136,...,0.00000,0.000000,0.000000,0.005682,0.005682,0.994318,0.676136,0.323864,8,6
723f73c85e91fc31d147dfade389d4f9,0,0.034000,0.0,0.002000,0.0,0.03400,0.044000,0.078000,0.004000,0.018000,...,0.07000,0.082000,0.134000,0.074000,0.000000,1.000000,0.724000,0.276000,7,2
54209126056016c7c391c0c8fd8e6eff,0,0.013746,0.0,0.000000,0.0,0.00000,0.024055,0.079038,0.006873,0.020619,...,0.00000,0.000000,0.199313,0.226804,0.000000,1.000000,0.756014,0.243986,0,1


In [3]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
Index: 130 entries, e6762ba2ffbca07ab6cee7551caeaad5 to 1c234775cae774823f38abe6721e61a4
Data columns (total 28 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   cat_code                                                  130 non-null    int64  
 1   source_channel_quantity_152bf0ce464047b9499ccb9e5b9b77a8  130 non-null    float64
 2   source_channel_quantity_3ab2427543039f8c9f17d06f6f65a3a7  130 non-null    float64
 3   source_channel_quantity_5a97b8efd901c1d28ff86522b95babb9  130 non-null    float64
 4   source_channel_quantity_67c19e107de33cab7ea9a9db8bc9ccd2  130 non-null    float64
 5   source_channel_quantity_7261d300057219056592010c7bdaf5ee  130 non-null    float64
 6   source_channel_quantity_98defd6ee70dfb1dea416cecdf391f58  130 non-null    float64
 7   source_channel_quantity_9d3e0fcbc1f16d80a76026e8f1c2

In [4]:
df_sales = pd.read_csv('../data/clean_step2.csv')
df_sales.head()

,order_id,code,quantity,price,pis_cofins,icms,tax_substitution,category,liquid_cost,order_status,...,perc_icms,perc_tax_substitution,unit_revenue,unit_price,unit_liquid_price,unit_perc_pis_cofins,unit_perc_icms,unit_perc_tax_substitution,unit_liquid_cost,unit_markup
0,bcb59c839e78b2601374cbad9239ca7b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.000,191.8416,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.60,537.4868,978.90,1080.1933,9.25,0.0,19.60,542.7065,1.990382
1,88eb0ac86af1a521c0831298d22dea8b,e6762ba2ffbca07ab6cee7551caeaad5,1,978.90,90.5483,0.000,191.8416,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.60,537.4868,978.90,1080.1933,9.25,0.0,19.60,542.7065,1.990382
2,1c175bc61b9b659bbf011b2e5e3dcec6,e6762ba2ffbca07ab6cee7551caeaad5,1,976.05,90.2846,0.000,192.3325,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,0.0,19.71,535.3914,976.05,1078.0979,9.25,0.0,19.71,542.7065,1.986521
3,a8ad36828898fa3f6efeb5bd19c076f2,e6762ba2ffbca07ab6cee7551caeaad5,1,1089.10,100.7418,185.147,0.0000,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,17.0,0.00,260.5047,1089.10,803.2112,9.25,17.0,0.00,542.7065,1.480010
4,9cbfaac6e04ea6ed454b843c94f2c29c,e6762ba2ffbca07ab6cee7551caeaad5,1,949.00,87.7825,170.820,0.0000,4ece547755cba9e7fc14125bc895f31b,542.7065,entrega total,...,18.0,0.00,147.6910,949.00,690.3975,9.25,18.0,0.00,542.7065,1.272138


In [5]:
df_sales.columns

Index(['order_id', 'code', 'quantity', 'price', 'pis_cofins', 'icms',
       'tax_substitution', 'category', 'liquid_cost', 'order_status',
       'capture_date', 'process_date', 'process_status', 'source_channel',
       'liquid_price', 'revenue', 'markup', 'week_number', 'month',
       'perc_pis_cofins', 'perc_icms', 'perc_tax_substitution', 'unit_revenue',
       'unit_price', 'unit_liquid_price', 'unit_perc_pis_cofins',
       'unit_perc_icms', 'unit_perc_tax_substitution', 'unit_liquid_cost',
       'unit_markup'],
      dtype='object')

In [6]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144731 entries, 0 to 144730
Data columns (total 30 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   order_id                    144731 non-null  object 
 1   code                        144731 non-null  object 
 2   quantity                    144731 non-null  int64  
 3   price                       144731 non-null  float64
 4   pis_cofins                  144731 non-null  float64
 5   icms                        144731 non-null  float64
 6   tax_substitution            144731 non-null  float64
 7   category                    144731 non-null  object 
 8   liquid_cost                 144731 non-null  float64
 9   order_status                144731 non-null  object 
 10  capture_date                144731 non-null  object 
 11  process_date                144731 non-null  object 
 12  process_status              144731 non-null  object 
 13  source_channel

## Incorporating some feature insights from previous EDAs

* perc_pis_cofins - Helps to understand the type of product. 
* source_channel - indicate some movement, there are 5 top channels that might help creating diversity (5 times more instances to predict. 
* hc_cluster and kmeans_cluster, will act as an embedding of seasonal and source behaviours. 
* markup/revenue, I will use the markup since it is a function of the liquid cost and reflect the revenue. Both a are f(liquid_cost) and quantity = g(f(liquid_cost)) ==> demand = g(price) 
* sum(quantity) - will be the Y 
* I'll also make a quantization of the markup to guarantee a reasoable cardinality 


In [7]:
df_sales['year_month'] = df_sales['capture_date'].apply(lambda x: x[:7])

In [8]:
df_dataset = df_sales.groupby(['code', 'year_month', 'month', ]).agg({'quantity':'sum'})

In [9]:
df_dataset = df_dataset.join(df_products[['hc_cluster', 'kmeans_cluster','cat_code']], on='code', how='left')

In [10]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1396 entries, ('0671c2b9132a3f5215a4212ce0691694', '2016-11', 11) to ('ffdad3ddbaf6c76c9bba1b48c51e03c6', '2017-06', 6)
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   quantity        1396 non-null   int64
 1   hc_cluster      1396 non-null   int64
 2   kmeans_cluster  1396 non-null   int64
 3   cat_code        1396 non-null   int64
dtypes: int64(4)
memory usage: 50.5+ KB


In [11]:
df_dataset = df_dataset.reset_index()
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1396 entries, 0 to 1395
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   code            1396 non-null   object
 1   year_month      1396 non-null   object
 2   month           1396 non-null   int64 
 3   quantity        1396 non-null   int64 
 4   hc_cluster      1396 non-null   int64 
 5   kmeans_cluster  1396 non-null   int64 
 6   cat_code        1396 non-null   int64 
dtypes: int64(5), object(2)
memory usage: 76.5+ KB


## Checking cardinalities
The idea is to keep those at some levels during the train, validation and test separations

In [12]:
df_dataset.groupby(['code']).month.nunique().reset_index().describe()

,month
count,130.000000
mean,10.276923
std,2.369101
min,1.000000
25%,9.000000
50%,11.000000
75%,12.000000
max,12.000000


In [13]:
df_dataset.describe()

,month,quantity,hc_cluster,kmeans_cluster,cat_code
count,1396.000000,1396.000000,1396.000000,1396.000000,1396.000000
mean,6.318052,108.926934,2.388968,4.014327,0.814470
std,3.471418,235.188518,2.047429,3.265955,2.092305
min,1.000000,1.000000,1.000000,0.000000,0.000000
25%,3.000000,12.000000,1.000000,0.000000,0.000000
50%,6.000000,36.000000,2.000000,7.000000,0.000000
75%,9.000000,90.250000,2.000000,7.000000,0.000000
max,12.000000,2063.000000,9.000000,8.000000,10.000000


In [14]:
df_dataset['Y'] = df_dataset.quantity
df_dataset = df_dataset.drop(columns='quantity')

## Converting to dummies the categorical columns

In [15]:
cat_columns = ['hc_cluster', 'kmeans_cluster', 'cat_code']
for col in cat_columns:     
    df_dataset[col] = df_dataset[col].astype('category')

In [16]:
df_dataset_v2 = pd.get_dummies(df_dataset, columns=cat_columns)

In [17]:
df_dataset_v2.columns

Index(['code', 'year_month', 'month', 'Y', 'hc_cluster_1', 'hc_cluster_2',
       'hc_cluster_3', 'hc_cluster_4', 'hc_cluster_5', 'hc_cluster_6',
       'hc_cluster_7', 'hc_cluster_8', 'hc_cluster_9', 'kmeans_cluster_0',
       'kmeans_cluster_1', 'kmeans_cluster_2', 'kmeans_cluster_3',
       'kmeans_cluster_4', 'kmeans_cluster_5', 'kmeans_cluster_6',
       'kmeans_cluster_7', 'kmeans_cluster_8', 'cat_code_0', 'cat_code_1',
       'cat_code_2', 'cat_code_3', 'cat_code_4', 'cat_code_5', 'cat_code_6',
       'cat_code_7', 'cat_code_8', 'cat_code_9', 'cat_code_10'],
      dtype='object')

In [18]:
df_dataset_v2.head()

,code,year_month,month,Y,hc_cluster_1,hc_cluster_2,hc_cluster_3,hc_cluster_4,hc_cluster_5,hc_cluster_6,...,cat_code_1,cat_code_2,cat_code_3,cat_code_4,cat_code_5,cat_code_6,cat_code_7,cat_code_8,cat_code_9,cat_code_10
0,0671c2b9132a3f5215a4212ce0691694,2016-11,11,664,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0671c2b9132a3f5215a4212ce0691694,2016-12,12,225,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0671c2b9132a3f5215a4212ce0691694,2017-01,1,211,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0671c2b9132a3f5215a4212ce0691694,2017-02,2,634,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0671c2b9132a3f5215a4212ce0691694,2017-03,3,1343,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
df_dataset_v2.to_csv('../data/forecast_dataset_exp11.csv', index=False)

## Splitting the Sets
* We will use the temporal series cross validation
* 3 months minimum intervals
* 3 training, 6 validation, 3 test
* 4 training, 5 validation, 3 test
* 5 training, 4 validation, 3 test
* 6 training, 3 validation, 3 test

In [20]:
df_sets = df_dataset_v2[['year_month', 'month']].drop_duplicates().sort_values(['year_month', 'month'])

In [21]:
test_months = df_sets['year_month'].tail(4).values[:3]

In [22]:
test_months

array(['2017-03', '2017-04', '2017-05'], dtype=object)

In [23]:
train_val = []
for i in range(7): 
    sets_year_month = list(df_sets['year_month'].head(9).values)
    train_val.append({
        'set': i,
        'train': sets_year_month[:3+i], 
        'val'  : list(test_months) if i ==6 else sets_year_month[3+i:], 
    })
    print(train_val[-1])

{'set': 0, 'train': ['2016-06', '2016-07', '2016-08'], 'val': ['2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 1, 'train': ['2016-06', '2016-07', '2016-08', '2016-09'], 'val': ['2016-10', '2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 2, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10'], 'val': ['2016-11', '2016-12', '2017-01', '2017-02']}
{'set': 3, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11'], 'val': ['2016-12', '2017-01', '2017-02']}
{'set': 4, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12'], 'val': ['2017-01', '2017-02']}
{'set': 5, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01'], 'val': ['2017-02']}
{'set': 6, 'train': ['2016-06', '2016-07', '2016-08', '2016-09', '2016-10', '2016-11', '2016-12', '2017-01', '2017-02'], 'val': ['2017-03', '2017-04', '2017-05']}


In [24]:
df_train_val_test = pd.DataFrame(train_val)
df_train_val_test['reserved'] = df_train_val_test.set == 6

In [25]:
df_train_val_test.to_csv('../data/train_val_test.csv', index=False)